# Technical Factor - Vertical Horizontal Filter (VHF)

In [5]:
import dai
import pandas as pd

In [6]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [7]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (m_max(close, 28) - m_min(close, 28)) / m_sum(ABS(close - m_lag(close, 1)), 28) AS VHF,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(VHF > m_avg(VHF, 28), 1, 0) AS TRBY1,
        IF(VHF < m_avg(VHF, 28), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(VHF > m_lag(VHF, 1) AND m_lag(VHF, 1) >= m_lag(VHF, 2), 1, 0) AS MTBY1,
        IF(VHF < m_lag(VHF, 1) AND m_lag(VHF, 1) <= m_lag(VHF, 2), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(VHF < m_avg(VHF, 28) AND (VHF - m_lag(VHF, 1)) > 0, 1, 0) AS RVBY1,
        IF(VHF > m_avg(VHF, 28) AND (VHF - m_lag(VHF, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(VHF, 1) <= m_lag(m_avg(VHF, 28), 1) AND VHF > m_avg(VHF, 28), 1, 0) AS BKBY1,
        IF(m_lag(VHF, 1) >= m_lag(m_avg(VHF, 28), 1) AND VHF < m_avg(VHF, 28), 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        VHF,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [8]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,VHF,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-19,600519.SH,0.244944,0,0,0,0,0,0,0,0
1,2025-02-20,600519.SH,0.242222,0,0,0,0,0,0,0,0
2,2025-02-21,600519.SH,0.257068,0,0,0,0,0,0,0,0
3,2025-02-24,600519.SH,0.250467,0,0,0,0,0,0,0,0
4,2025-02-25,600519.SH,0.235091,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
210,2025-12-25,600519.SH,0.427534,1,0,1,0,0,0,0,0
211,2025-12-26,600519.SH,0.410697,1,0,0,0,0,1,0,0
212,2025-12-29,600519.SH,0.371790,1,0,0,1,0,1,0,0
213,2025-12-30,600519.SH,0.351920,1,0,0,1,0,1,0,0
